# Loading and correcting data for RhFe 
# NanoLab experiments

In [2]:
%load_ext autoreload
%autoreload 2
%gui tk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#Imports
# Embedded imports
import os, sys
import copy
import math
import matplotlib.pyplot as plt
# Third party imports
import numpy as np
import pandas as pd
import scipy as sp
# Temporary solution for local imports
sys.path.append('..')
from xpstool import helpers 
from xpstool import Experiment, Spectrum, Region
from xpstool import Fitter

In [26]:
# ---SPECS DATA FOLDERS---
# Full path to data files
data_folder = "/Users/Shipilin/Documents/07_DataAnalysis/2018-RhFe_alloys/2018-November_Rh80Fe20/2018-11_Rh80Fe20_XPS/XY_RhFe"
data_subfolders = [
    "/Rh80Fe20(111)",
    "/Rh80Fe20(111)_7_11",
    "/Rh80Fe20(111)_8_11",
    "/Rh80Fe20(111)_9_11"
]
# Full path to output files
output_folder = "/Users/Shipilin/Documents/07_DataAnalysis/2018-RhFe_alloys/2018-November_Rh80Fe20/2018-11_Rh80Fe20_XPS/ProcessedData"

file_names = []
file_paths = []
file_idx = 0
for subfolder in data_subfolders:
    files = os.listdir("".join([data_folder, subfolder]))
    files = sorted(files, key=lambda x: int(x.rpartition('.')[0].rpartition('_')[2]))
    for file in files:
        if file.endswith(".xy"): 
            file_names.append(file)
            file_paths.append("/".join(["".join([data_folder, subfolder]), file]))
            #print(f"---> file[{file_idx}]: {file}")
            file_idx += 1

In [47]:
# Loading all spectra to a list
all_spectra = []
for path in file_paths:
    all_spectra.append(Spectrum(path=path, file_type="specs"))
       
print(f"{len(all_spectra)} spectra were loaded successfuly.")

# Check for spectra that are empty or contain more than one region
for spectrum in all_spectra:
    if spectrum.isEmpty():
        print(f"{spectrum.getID()} spectrum: 0 regions")
    if len(spectrum.getRegions()) > 1:
        print(f"{spectrum.getID()} spectrum: {len(spectrum.getRegions())} regions")

224 spectra were loaded successfuly.
Rh80Fe20(111)_3 spectrum: 0 regions
Rh80Fe20(111)_7_11_1 spectrum: 0 regions
Rh80Fe20(111)_7_11_2 spectrum: 0 regions
Rh80Fe20(111)_7_11_3 spectrum: 0 regions
Rh80Fe20(111)_7_11_60 spectrum: 0 regions
Rh80Fe20(111)_8_11_1 spectrum: 0 regions
Rh80Fe20(111)_8_11_2 spectrum: 0 regions
Rh80Fe20(111)_8_11_3 spectrum: 0 regions
Rh80Fe20(111)_9_11_1 spectrum: 0 regions
Rh80Fe20(111)_9_11_2 spectrum: 0 regions
Rh80Fe20(111)_9_11_3 spectrum: 0 regions
Rh80Fe20(111)_9_11_48 spectrum: 0 regions
Rh80Fe20(111)_9_11_49 spectrum: 0 regions
Rh80Fe20(111)_9_11_50 spectrum: 0 regions
Rh80Fe20(111)_9_11_51 spectrum: 0 regions
Rh80Fe20(111)_9_11_52 spectrum: 0 regions


In [48]:
for spectrum in all_spectra:
    if not spectrum.isEmpty():
        print(spectrum.getID())
        print(f"--->{spectrum.getRegion().getID()}")

Rh80Fe20(111)_1
--->Rh80Fe20(111)_1:Rh3d align (NE)
Rh80Fe20(111)_2
--->Rh80Fe20(111)_2:Rh3d align (GE)
Rh80Fe20(111)_4
--->Rh80Fe20(111)_4:survey
Rh80Fe20(111)_5
--->Rh80Fe20(111)_5:Survey after 2 cleaning cycles
Rh80Fe20(111)_6
--->Rh80Fe20(111)_6:Fe 2p  after 2 cleaning cycles
Rh80Fe20(111)_7
--->Rh80Fe20(111)_7:Rh 3d  after 2 cleaning cycles
Rh80Fe20(111)_8
--->Rh80Fe20(111)_8:C 1s  after 2 cleaning cycles
Rh80Fe20(111)_9
--->Rh80Fe20(111)_9:O 1S + Rh 3P  after 2 cleaning cycles
Rh80Fe20(111)_10
--->Rh80Fe20(111)_10:EF after 2 cleaning cycles
Rh80Fe20(111)_11
--->Rh80Fe20(111)_11:Survey after 2 cleaning cycles (spot 2)
Rh80Fe20(111)_12
--->Rh80Fe20(111)_12:Survey after 2 cleaning cycles (spot 1) (with window open...)
Rh80Fe20(111)_13
--->Rh80Fe20(111)_13:Survey after 2 cleaning cycles (spot 3)
Rh80Fe20(111)_14
--->Rh80Fe20(111)_14:survey (300C)
Rh80Fe20(111)_15
--->Rh80Fe20(111)_15:Fe 2p  300C
Rh80Fe20(111)_16
--->Rh80Fe20(111)_16:Rh 3d  300C
Rh80Fe20(111)_17
--->Rh80Fe20(111)_17:C